In [28]:
import sys
sys.path.append("../")
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from text_anomalies.model import VAE, SSVAE


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(1000,300, device=device)
model2 = SSVAE(
    vocab_size=1000,
    y_size=5,
    embedding_size=300,
    hidden_size=256,
    latent_size=16,
    device=device,
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print(model2)

SSVAE(
  (embedding): Embedding(1000, 300)
  (encoder): Encoder(
    (rnn): RNN(300, 256, num_layers=2, batch_first=True)
  )
  (pre_decoder): Sequential(
    (0): Linear(in_features=21, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (latent_to_hidden): Linear(in_features=16, out_features=512, bias=True)
    (rnn): RNN(300, 256, num_layers=2, batch_first=True)
    (output): Linear(in_features=256, out_features=1000, bias=True)
  )
  (y_predict): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=5, bias=True)
  )
  (lambd): Lambda(
    (hidden_to_mean): Linear(in_features=256, out_features=16, bias=True)
    (hidden_to_logvar): Linear(in_features=256, out_featur

In [29]:
# Test
input = np.array([10,10,12,12,34,12,0,0,0,0,0])
input = np.stack([input, input,input])
input = torch.from_numpy(input)
# Stack input
print(input.shape)
print(model)
model2.to("cuda")

output = model2(input.to("cuda"))

torch.Size([3, 11])
VAE(
  (embedding): Embedding(1000, 300)
  (input_highway): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=240, bias=True)
    (3): ReLU()
    (4): Linear(in_features=240, out_features=180, bias=True)
    (5): ReLU()
    (6): Linear(in_features=180, out_features=150, bias=True)
    (7): ReLU()
  )
  (encoder): Encoder(
    (rnn): RNN(150, 256, num_layers=2, batch_first=True)
  )
  (lambd): Lambda(
    (hidden_to_mean): Linear(in_features=256, out_features=16, bias=True)
    (hidden_to_logvar): Linear(in_features=256, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (latent_to_hidden): Linear(in_features=16, out_features=512, bias=True)
    (rnn): RNN(300, 256, num_layers=2, batch_first=True)
    (output): Linear(in_features=256, out_features=1000, bias=True)
  )
)


In [17]:
# Get mostlikely output[0] [3, 11, 1000] -> [3, 11]
tokens = torch.argmax(output[0], dim=2)
tokens

tensor([[495, 829, 747, 232, 747, 747, 747, 747, 747, 747, 747],
        [365, 502, 817, 300, 747, 747, 747, 747, 747, 747, 747],
        [495, 300, 747, 232, 747, 747, 747, 747, 747, 747, 747]],
       device='cuda:0')

In [23]:
torch.topk(output[0].exp(), dim=2, k=2)[1].shape

torch.Size([3, 11, 2])

In [31]:
output = model2(input.to("cuda"))
tokens = torch.argmax(output[0], dim=2)
tokens

tensor([[653, 462, 445, 445, 302, 302, 302, 445, 445, 445, 445],
        [ 47, 684, 302, 445, 302, 445, 302, 445, 445, 445, 445],
        [653, 462, 445, 445, 302, 302, 302, 445, 445, 445, 445]],
       device='cuda:0')